In [ ]:
#hidden cell to be executed BEFORE the presentation
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import dftpy
from dftpy.ions import Ions
from dftpy.field import DirectField
from dftpy.grid import DirectGrid
from dftpy.functional import LocalPseudo, Functional, TotalFunctional
from dftpy.formats import io
from dftpy.math_utils import ecut2nr
from dftpy.time_data import TimeData
from dftpy.optimization import Optimization
from dftpy.mpi import sprint
from IPython.lib.display import YouTubeVideo
from IPython.display import IFrame
from ase.visualize import view
PP_list = {'Al': 'Al_lda.oe01.recpot'}
#import fortecubeview

<center>
    <h1>Introduction to DFT</h1>
    <h3> Kohn-Sham and orbital-free</h3>
<center>
<br>
<table>
  <tr>
      <td><p><h1>Team Rutgers</h1></p><p><h2>Rutgers University-Newark</h2></p></td>
      <td><img src="figures/logos/run.png" width=300 height=300 /></td>
  </tr>
  <tr>
    <td></td>
    <td> http://prg.rutgers.edu</td>
  </tr>
</table>
<br>
<h3>Retrieve this presentation at:</h3>
<br>
<center>/ocean/projects/che240027p/shared/day1/slides/QMS_2024_DFT-1.ipynb</center>

<br>
<br>

#### Quantum Multiscale School -- Boise, Idaho -- June 24-28, 2024


# Goals of this lecture + hands-on session
- Reasons for considering DFT
- Basics of the theory behind DFT, Orbital-Free DFT and Kohn-Sham DFT
- KS equations
- Basis sets: plane waves, Gaussian-type orbitals
- Pseudopotentials (local part)
- Sample of OF-DFT and KS-DFT simulations

# The Real World
<table>
    <tr>
      <td><h3>Photocatalyst</h3></td>
        <td><h3>Catalytic nanoparticles</h3></td>
  </tr>
  <tr>
      <td><img src="figures/science/photocatalyst.png" height=500 /></td>
      <td><img src="figures/science/catalyst.png" height=500 /></td>
  </tr>
    <tr>
        <td>Chem. Comm., 43, 6551 (2009)</td>
        <td>PCCP, 21, 15080 (2019)</td>
    </tr>
</table>   

# Available electronic structure methods
<br>
<center>
    <img src="figures/science/electronic_structure.png" width=1600 />
</center>

# Challenge 1 (5 minutes!)

[Let's check your QM/DFT knowledge.](https://rutgers.ca1.qualtrics.com/jfe/form/SV_dbuLoAqBETcyZts) 
- Write the molecular Hamiltonian identifying the main contributions.
- Write the electron-nuclear attraction potential using $r_i$ and $R_\alpha$ for the electron and nuclear coordinates, respectively.
- Write the Kohn-Sham DFT energy density functional and the associated noninteracting kinetic energy.

<center>
    <img src="figures/random/qr_poll1.png" width=350 />
</center>


# The Hohenberg and Kohn theorems
<br>
<br>
$$
\Psi_0 \leftrightarrow n(r) \leftrightarrow v_{eN}(r)
$$

Therefore $n(r)$, $v_{eN}(r)$ or $\Psi_0$ hold the same information. 

In particular:

$$
E \equiv E[\Psi_0] \equiv E[v_{eN}] \equiv E[n]
$$

DFT exploits the latter as follows:

$$
E[n] = T[n] + E_{ee}[n]+E_{eN}[n]+E_{NN}
$$

<br>
<br>
<br>
<center>
<span style="font-size:45pt;"><i>               n(r)</i></span>
</center>
<br>
<br>
<br>
<center>...the density determines everything...</center>

# DFT: a theory of maps
<br>
<br>
<center>
    <img src="figures/science/maps.png" width=800 />
</center>

# DFT: a theory of maps
<br>
<center>
    <img src="figures/science/maps2.png" width=1200 />
</center>

# DFT energy functionals for the 2 useful fictitious systems

### Single-particle or "Kohn-Sham" DFT

$$
E[n] = T_s[n] + E_{H}[n] + E_{xc}[n] + \int v_{eN}[n](r) n(r) dr  + E_{NN}
$$

where the density $n(r) = \sum_i |\phi_i(r)|^2$. And the single-particle kinetic energy is

$$
T_{s}[n] \equiv T_s[\{\phi_i\}]=  -\frac{1}{2}\sum_i n_i \langle \phi_i | \nabla^2 | \phi_i\rangle = -\frac{1}{2}\sum_i n_i \int \phi_i^*(r) \nabla^2 \phi_i(r) dr
$$

Mind: $T_s \neq T$.



The e-e repulsion and the total kinetic energy are related to $T_s$ and $E_{xc}$ as follows:

$$
E_H[n]=\frac{1}{2}\int \frac{n(r)n(r')}{|r-r'|}drdr'
$$

$$
E_{xc} = \text{Approximated!} \to T[n] + E_{ee}[n] = T_s[n] + E_H[n] + E_{xc}[n]
$$

# DFT energy functionals for the 2 useful fictitious systems

### Orbital-free DFT

$$
E[n] = T_{B}[n] + T_P[n] + E_{H}[n] + E_{xc}[n] + \int v_{eN}[n](r) n(r) dr  + E_{NN}
$$

where

$$
T_{B}[n] = T_{vW}[n] = -\frac{1}{2}\int \phi^*(r) \nabla^2 \phi(r) dr
$$

where: $\phi(r)=\sqrt{n(r)}$, and 

$$
T_P[n] = T_{s}[n] - T_{vW}[n]
$$

# Solving for the electronic structure

### KS equations for KS and OF-DFT

Define an appropriate Lagrangian:
$$
\mathcal{L}_{KS}[\{\phi_i\}] = E[\{\phi_i\}] - \sum_{ij} \varepsilon_{ij}\left(\langle \phi_j|\phi_i \rangle - \delta_{ij}\right), \qquad \text{for KS-DFT}
$$

Define an appropriate Lagrangian:
$$
\mathcal{L}_{OF}[n] = E[n] -  \mu \left( \int n(r)dr - N\right), \qquad \text{for OF-DFT}
$$

### Let's minimize the Lagrangians to find the ground state KS orbitals and density

For KS-DFT, imposing $\frac{\delta \mathcal{L}_{KS}[\{\phi_i\}]}{\delta \langle \phi_j|}=0$ or just $\frac{\delta \mathcal{L}_{KS}[\{\phi_i\}]}{\delta \phi_j^*(r)}=0$, and choosing the so-called <i>canonical</i> orbitals (i.e., $\varepsilon_{ij}=\varepsilon_{i}\delta_{ij}$),

For OF-DFT, imposing $\frac{\delta \mathcal{L}_{OF}[n]}{\delta \langle \phi|}=0$ or just $\frac{\delta \mathcal{L}_{OF}[n]}{\delta \phi^*(r)}=0$, where $\phi(r) = \sqrt{n(r)}$,

we reach the so-called <span style="color: red;">Kohn-Sham equations</span>:

$$
-\frac{1}{2}\nabla^2 \phi_i(r) + v_s[n](r)\phi_i(r) = \varepsilon_i\phi_i(r), \qquad \text{for KS-DFT}
$$

$$
-\frac{1}{2}\nabla^2 \phi(r) + v_B[n](r)\phi(r) = \mu\phi(r), \qquad \text{for OF-DFT}
$$


# Challenge 2 (10 minutes!)
[Link to the poll](https://rutgers.ca1.qualtrics.com/jfe/form/SV_6LIHtaXrwnoe6LY)

Considering the chain rule of functional differentiation (analogous to regular differentiation where $n(r)$ is treated as the variable and $\phi^*$ and $\phi$ are independent variables):

$$
\frac{\delta F[n]}{\delta \phi_j^*(r)} = \int \frac{\delta F[n]}{\delta n(r')}\frac{\delta n(r')}{\delta \phi_j^*(r)}dr' = \frac{\delta F[n]}{\delta n(r)} \phi_j(r).
$$


1) Derive the KS equations and show that the KS potential is given by:

$$
v_s[n](r) = \frac{\delta E_{H}[n]}{\delta n(r)} + \frac{\delta E_{xc}[n]}{\delta n(r)} + v_{eN}(r)
$$


2) <span style="color: red;">OPTIONAL</span>: Show that the KS potential of OF-DFT is:

$$
v_B[n](r) = \underbrace{\frac{\delta T_{s}[n]}{\delta n(r)} - \frac{\delta T_{vW}[n]}{\delta n(r)}}_{\frac{\delta T_P[n]}{\delta n(r)}} + \frac{\delta E_{H}[n]}{\delta n(r)} + \frac{\delta E_{xc}[n]}{\delta n(r)} + v_{eN}(r)
$$
reaching the KS equation: $-\frac{1}{2}\nabla^2 \sqrt{n(r)} + v_B[n](r)\sqrt{n(r)} = \mu\sqrt{n(r)}$. Where $\mu$ has the same meaning of $\varepsilon_i$ for the KS case.

# Solvers
<br>
<br>
<table border="1" style="width:100%; text-align:center;">
    <tr>
        <th></th>
        <th style="text-align:center;border-right: 2px solid red;padding: 40px;">OF-DFT</th>
        <th style="text-align:center;">KS-DFT</th>
    </tr>
    <tr>
        <th style="text-align:center;padding: 40px;">Direct Minimization</th>
        <td style="text-align:center; border-right: 2px solid red; ">
            \( n_0(\mathbf{r})=\arg\underset{n}\min\big\{ \mathcal{L}_{OF}[n]\big\} \)
        </td>
        <td style="text-align:center;">
            \( \{\phi_i^0\} = \arg\underset{\{\phi_i\}}\min\big\{ \mathcal{L}_{KS}[\{\phi_i\}]\big\} \)
        </td>
    </tr>
    <tr>
        <th style="text-align:center;padding: 40px;">SCF</th>
        <td style="text-align:center;border-right: 2px solid red; ">
            \( -\frac{1}{2}\nabla^2 \sqrt{n(r)} + v_B(r)\sqrt{n(r)} = \mu\sqrt{n(r)}\)
        </td>
        <td style="text-align:center;">
            \( -\frac{1}{2}\nabla^2 \phi_i(r) + v_s(r)\phi_i(r) = \varepsilon_i\phi_i(r)\)
        </td>
    </tr>
</table>


# Solvers
<br>
<br>
<table border="1" style="width:100%; text-align:center;">
    <tr>
        <th></th>
        <th style="text-align:center;border-right: 2px solid red;padding: 40px;">OF-DFT</th>
        <th style="text-align:center;">KS-DFT</th>
    </tr>
    <tr>
        <th style="text-align:center;padding: 40px;">Direct Minimization</th>
        <td style="text-align:center; border-right: 2px solid red; background-color: lightgreen;">
            \( n_0(\mathbf{r})=\arg\underset{n}\min\big\{ \mathcal{L}_{OF}[n]\big\} \)
        </td>
        <td style="text-align:center;">
            \( \{\phi_i^0\} = \arg\underset{\{\phi_i\}}\min\big\{ \mathcal{L}_{KS}[\{\phi_i\}]\big\} \)
        </td>
    </tr>
    <tr>
        <th style="text-align:center;padding: 40px;">SCF</th>
        <td style="text-align:center;border-right: 2px solid red; ">
            \( -\frac{1}{2}\nabla^2 \sqrt{n(r)} + v_B(r)\sqrt{n(r)} = \mu\sqrt{n(r)}\)
        </td>
        <td style="text-align:center;background-color: lightgreen;">
            \( -\frac{1}{2}\nabla^2 \phi_i(r) + v_s(r)\phi_i(r) = \varepsilon_i\phi_i(r)\)
        </td>
    </tr>
</table>


# Basis sets (discretization)

To be able to run simulations, we need to discretize the space in which the KS and OF wavefunctions / densities live. We will consider plane waves (PW) and Gaussian-type orbitals (GTOs). The general idea is the following:
$$
\phi_i(r) = \sum_\mu^M c_\mu \chi_\mu(r) \text{, where } \chi_\mu \text{ are basis functions}
$$

<table border="1" style="width:100%; text-align:center;">
    <tr>
        <th></th>
        <th style="text-align:center;border-right: 2px solid red;padding: 40px;">GTOs</th>
        <th style="text-align:center;">PW</th>
    </tr>
    <tr>
        <th style="text-align:center;padding: 40px;">Definition</th>
        <td style="text-align:center; border-right: 2px solid red;">
            \( \chi_\mu(r) = x^{i_\mu}y^{j_\mu}z^{k_\mu} e^{-\frac{|r-R_\mu|^2}{2\sigma_\mu^2}} \)
        </td>
        <td style="text-align:center;">
            \( \chi_\mu(r) = \frac{1}{\sqrt{\Omega}} e^{i G_\mu \cdot r},\,\,G_\mu = \left(\frac{2\pi\, i_\mu}{a},\frac{2\pi\, j_\mu}{b},\frac{2\pi\, k_\mu}{c}\right)\)
        </td>
    </tr>
    <tr>
        <th style="text-align:center;padding: 40px;">Location</th>
        <td style="text-align:center;border-right: 2px solid red;">
            \( R_\mu \in \text{centers of atoms}\)
        </td>
        <td style="text-align:center;">
            Spread out throughout the simulation cell
        </td>
    </tr>
    <tr>
        <th style="text-align:center;padding: 40px;">Number of functions</th>
        <td style="text-align:center;border-right: 2px solid red;">
            \( \simeq 10 \) per atom
        </td>
        <td style="text-align:center;">
            \( \propto \Omega = a \cdot b \cdot c \), generally \( 10^4 \) to \( 10^6 \)
        </td>
    </tr>
    <tr>
        <th style="text-align:center;padding: 40px;">Handling eN potential</th>
        <td style="text-align:center;border-right: 2px solid red;">
            <center><img src="./figures/science/fullpot.png" alt="GTOs image" width=400 ></center>
        </td>
        <td style="text-align:center;">
            <center><img src="./figures/science/pseudopot.png" alt="PW image" width=400 ></center>
        </td>
    </tr>
</table>


# Pseudo potentials cannot incorporate core electrons

<table border="1" style="width:100%; text-align:center;">
    <tr>
        <th><center><img src="./figures/science/Electron-Configuration.jpg" alt="econf" width=400 ></center></th>
        <th><center><img src="./figures/science/pseudo_core.png" alt="econf" width=400 ></center></th>
    </tr>
</table>






# Challenge 3 (3 minutes!)

1) Download a pseudopotential from your favorite library
2) Determine the electronic configuration of the ion

In [ ]:
additional_files = {'C.pbe-n-rrkjus_psl.1.0.0.UPF' : 'https://pseudopotentials.quantum-espresso.org/upf_files/C.pbe-n-rrkjus_psl.1.0.0.UPF'}
from dftpy.formats import download_files
download_files(additional_files)
!cat C.pbe-n-rrkjus_psl.1.0.0.UPF

# Let's start easy: OF-DFT simulations with DFTpy

### First generate a model system: bulk Al (cubic cell)

In [ ]:
from ase.build import bulk
atoms = bulk('Al', 'fcc', a=4.05, cubic=True)
ions = Ions.from_ase(atoms)
view(ions, viewer='ngl')

### Then run a OF-DFT simulation with `DFTpy`

In [ ]:
additional_files = {'al.lda.recpot' : 'https://raw.githubusercontent.com/EACcodes/local-pseudopotentials/master/BLPS/LDA/reci/al.lda.recpot'}
download_files(additional_files)

In [ ]:
grid = DirectGrid(ecut=20.0, lattice=ions.cell) #ecut 20 Ha = 40 Ry
PSEUDO = LocalPseudo(grid = grid, ions=ions, PP_list={'Al': 'al.lda.recpot'})
rho_ini = DirectField(grid=grid)
rho_ini[:] = ions.get_ncharges()/ions.cell.volume
HARTREE = Functional(type='HARTREE')
XC = Functional(type='XC',name='LDA')

#### DFTpy uses plane waves (PW)
<center><img src="./figures/science/PW_explained.png" alt="PW" width=700 ></center>

### For OF-DFT only: select a kinetic energy functional

- $T_s[n]$ also called KEDF. In this example we use Thomas Fermi + von Weizsacker:
$$T_s[n] = \int C_{TF} n^{5/3}(\mathbf{r}) + \frac{\nabla n^{1/2}(\mathbf{r})\cdot \nabla n^{1/2}(\mathbf{r})}{2} d\mathbf{r}$$

In [ ]:
KE = Functional(type='KEDF',name='x_TF_y_vW')

### Define "total energy" and minimize $E[n]$

In [ ]:
evaluator = TotalFunctional(KE=KE, XC=XC, HARTREE=HARTREE, PSEUDO=PSEUDO)
optimization_options = {'econv' : 1e-6*ions.nat}
opt = Optimization(EnergyEvaluator=evaluator, optimization_options = optimization_options,
        optimization_method = 'TN')
%timeit -n1 -r1 rho = opt.optimize_rho(guess_rho=rho_ini)

<div class="alert alert-block alert-success"><center><b>Opportunities:</b> Can manipulate rho as you wish - many functionalities are available in DFTpy</center></div>

In [ ]:
from dftpy.constants import environ
environ['LOGLEVEL'] = 3  # turn off the output
rho = opt.optimize_rho(guess_rho=rho_ini)
rho.write('rho.cube',ions=ions)
!ls -l | grep rho.cube

In [ ]:
import matplotlib.pyplot as plt
plt.plot(rho[0,10,:],label="rho")
plt.plot(XC(rho).potential[0,10,:],label="xc")
plt.legend()
plt.title('OF-DFT electron density');

...let's try to re-run it with a different $T_s[n]$ but same $E_{xc}[n]$.

### Options for $T_s[n]$ and $E_{xc}$: (semi)local and nonlocal functionals
<br>
<center>
    <img src="figures/science/local_nonlocal.png" width=1600 />
</center>

 - For $T_S$: Check out this <a href="">Chem. Rev. (2023)</a> summarizing all that is known about $T_s$ approximations
 - For $E_{xc}$: Check out John Perdew's ideas leading to <a href="https://journals.aps.org/prb/abstract/10.1103/PhysRevB.23.5048">LDA</a>, <a href="https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.77.3865">GGA</a> and <a href="https://www.annualreviews.org/content/journals/10.1146/annurev-physchem-062422-013259">meta-GGA</a>


### Re-run of OF-DFT with nonlocal $T_s[n]$

In [ ]:
KE = Functional(type='KEDF',name='WT')
evaluator = TotalFunctional(KE=KE, XC=XC, HARTREE=HARTREE, PSEUDO=PSEUDO)
optimization_options = {'econv' : 1e-5*ions.nat}
opt = Optimization(EnergyEvaluator=evaluator, optimization_options = optimization_options,
        optimization_method = 'TN')
%timeit -n1 -r1 rho = opt.optimize_rho(guess_rho=rho_ini)

<center><div class="alert alert-block alert-danger">
    Nonlocal $T_s[n]$ are somewhat slower than semilocal functionals
</div> 
    </center>

# How does OF-DFT compare with KS-DFT?

<h4 style="text-align: center;">We will run a Quantum ESPRESSO calculation of the same system using QEpy</h4>

In [ ]:
from qepy.driver import Driver
from qepy.io import QEInput

### QEpy's input file is a dictionary

In [ ]:
qe_options = {
    '&control': {
        'calculation': "'scf'",
        'pseudo_dir': "'./'",
    },
    '&system': {
        'ibrav' : 0,
        'degauss': 0.005,
        'ecutwfc': 30,
        'occupations': "'smearing'"
    },
    'atomic_species': ['Al  26.98 Al.pbe-nl-kjpaw_psl.1.0.0.UPF'],
    'k_points gamma': [],
}

In [ ]:
additional_files = {'Al.pbe-nl-kjpaw_psl.1.0.0.UPF' : 'http://pseudopotentials.quantum-espresso.org/upf_files/Al.pbe-nl-kjpaw_psl.1.0.0.UPF'}
download_files(additional_files)

In [ ]:
options = {
    '&electrons': {
        'mixing_beta': 0.5},
    'cell_parameters angstrom':[
        '0.     2.025  2.025',
        '2.025  0.     2.025',
        '2.025  2.025  0.   '],
    'atomic_positions crystal': ['Al    0.0  0.0  0.0'],
    'k_points automatic': ['4 4 4 1 1 1'],
}

### Initialize QEpy

In [ ]:
qe_options = QEInput.update_options(options, qe_options=qe_options)

In [ ]:
driver = Driver(qe_options=qe_options, logfile=True)
%timeit -n1 -r1 driver.scf() 

In [ ]:
driver.get_scf_error()

In [ ]:
driver.stop()

<center><div class="alert alert-block alert-danger">
    KS-DFT is <b>much</b> slower than nonlocal OF-DFT for this small system.
</div></center>

# What can OF-DFT do?

<center>
    <p style="font-size:100%; text-align:center;">Shao, Mi & Pavanello, Phys. Rev. B <b>104</b>, 045118 (2021)</p>
    <br>
    <img src="figures/science/ofdft_surfaces.png" width=1800 />
    <div class="alert alert-block alert-success">Excellent results for difficult semiconductors and metals!</div>
    <div class="alert alert-block alert-success">Check out Mi et al. Chem. Rev. (2023) for more details.</div>
</center>

# What else can OF-DFT do?
<br>
<center>
    <p style="font-size:100%; text-align:center;"> J. Phys. Chem. Lett. <b>12</b>, 4134 (2021) &#x25C6; WIREs: Comp. Mol. Sci. <b>11</b>, e1482 (2021)</p>
    <br>
    <img src="figures/science/ofdft_showoff.png" width=1600 />
    <div class="alert alert-block alert-danger">No molecules, no chemical reactions.</div>
</center>

# The end!

See you later for the DFTpy hands-on session.
 - Dynamics of liquid metals
 - Any dynamics and relaxation you'd like to set up
 - play around with densities (plot, manipulate,...)